In [ ]:
import pandas as pd
import yfinance as yf
from ta import add_all_ta_features
import finplot as fplt
import warnings 

warnings.simplefilter(action="ignore")

In [ ]:
ticker_symbol = "AMZN"
start_date = "2015-04-01"
end_date = "2021-05-08"

ticker_data = yf.Ticker(ticker_symbol)

df = ticker_data.history(start=start_date, end=end_date)



In [ ]:
#get previous and next day highs as columns in df
df["Previous Day High"] = df["High"].shift(periods=-1)
df["Next Day High"] = df["High"].shift(periods=1)
df["Day After Next High"] = df["High"].shift(periods=2)

#get previous and next day lows as columns in df
df["Previous Day Low"] = df["Low"].shift(periods=-1)
df["Next Day Low"] = df["Low"].shift(periods=1)
df["Day After Next Low"] = df["Low"].shift(periods=2)


#function to add True to rows that match the desired pattern
def higher_pattern(row):
    if (row["High"] > row["Previous Day High"]) & (row["Next Day High"] > row["High"]) & (row["Next Day High"] > row["Day After Next High"]):
        val = True
    else:
        val = False
    return val

#create column that shows whether or not a day is a match
df["higher_pattern"] = df.apply(higher_pattern, axis=1)

#function to add True to rows that match the desired pattern
def lower_pattern(row):
    if (row["Low"] < row["Previous Day Low"]) & (row["Next Day Low"] < row["Low"]) & (row["Next Day Low"] > row["Day After Next Low"]):
        val = True
    else:
        val = False
    return val

#create column that shows whether or not a day is a match
df["lower_pattern"] = df.apply(lower_pattern, axis=1)


#save high higher lower to Excel
df.to_excel("nick pattern.xlsx")


In [ ]:
#finplot chart
df.reset_index(level=0, inplace=True)


In [14]:
ax = fplt.create_plot(rows=1)
candles = df[["Date", "Open", "Close", "High", "Low"]]
fplt.candlestick_ochl(candles, ax=ax)

high_wicks = df["higher_pattern"]
df.loc[(high_wicks), 'marker'] = df['High']
fplt.plot(df['Date'], df['marker'], ax=ax, color='#4a5', style='^', legend='HH')

fplt.show()